# Question B1 (15 marks)

Real world datasets often have a mix of numeric and categorical features – this dataset is one example. To build models on such data, categorical features have to be encoded or embedded.

PyTorch Tabular is a library that makes it very convenient to build neural networks for tabular data. It is built on top of PyTorch Lightning, which abstracts away boilerplate model training code and makes it easy to integrate other tools, e.g. TensorBoard for experiment tracking.

For questions B1 and B2, the following features should be used:   
- **Numeric / Continuous** features: dist_to_nearest_stn, dist_to_dhoby, degree_centrality, eigenvector_centrality, remaining_lease_years, floor_area_sqm
- **Categorical** features: month, town, flat_model_type, storey_range



---



In [3]:
!pip install pytorch_tabular[extra]

zsh:1: no matches found: pytorch_tabular[extra]


In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

ModuleNotFoundError: No module named 'pytorch_tabular'

1.Divide the dataset (‘hdb_price_prediction.csv’) into train, validation and test sets by using entries from year 2019 and before as training data, year 2020 as validation data and year 2021 as test data.
**Do not** use data from year 2022 and year 2023.



In [ ]:
df = pd.read_csv('hdb_price_prediction.csv')

# YOUR CODE HERE
train_data = df[df['year'] <= 2019]
validation_data = df[df['year'] == 2020]
test_data = df[df['year'] == 2021]

2.Refer to the documentation of **PyTorch Tabular** and perform the following tasks: https://pytorch-tabular.readthedocs.io/en/latest/#usage
- Use **[DataConfig](https://pytorch-tabular.readthedocs.io/en/latest/data/)** to define the target variable, as well as the names of the continuous and categorical variables.
- Use **[TrainerConfig](https://pytorch-tabular.readthedocs.io/en/latest/training/)** to automatically tune the learning rate. Set batch_size to be 1024 and set max_epoch as 50.
- Use **[CategoryEmbeddingModelConfig](https://pytorch-tabular.readthedocs.io/en/latest/models/#category-embedding-model)** to create a feedforward neural network with 1 hidden layer containing 50 neurons.
- Use **[OptimizerConfig](https://pytorch-tabular.readthedocs.io/en/latest/optimizer/)** to choose Adam optimiser. There is no need to set the learning rate (since it will be tuned automatically) nor scheduler.
- Use **[TabularModel](https://pytorch-tabular.readthedocs.io/en/latest/tabular_model/)** to initialise the model and put all the configs together.

In [ ]:
# YOUR CODE HERE
# Define the target variable and the names of the continuous and categorical variables
target = ['resale_price']
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_cols = ['month', 'town', 'flat_model_type', 'storey_range']

# Define the data configuration
data_config = DataConfig(
    target=target,
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols
)

# Define the trainer configuration
trainer_config = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50
)

# Define the category embedding model configuration
model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50",
)

# Define Optimizer Config
optimizer_config = OptimizerConfig()

# Initialise the model and put all the configs together
model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

# Train the model
model.fit(train=train_data, validation=validation_data)

# Report the test RMSE error and the test R2 value that you obtained
results = model.evaluate(test_data)
predictions = model.predict(test_data)

3.Report the test RMSE error and the test R2 value that you obtained.



In [ ]:
# YOUR CODE & RESULT HERE
import math
from sklearn.metrics import r2_score, root_mean_squared_error

print(f"RMSE: {math.sqrt(root_mean_squared_error(test_data['resale_price'], predictions['resale_price_prediction']))}")
print(f"R2: {r2_score(test_data['resale_price'], predictions['resale_price_prediction'])}")

4.Print out the corresponding rows in the dataframe for the top 25 test samples with the largest errors. Identify a trend in these poor predictions and suggest a way to reduce these errors.



In [2]:
# YOUR CODE & RESULT HERE
test_data['predictions'] = predictions['resale_price_prediction']
test_data['error'] = (test_data['resale_price'] - test_data['predictions'])**2
top_largest_error = test_data.sort_values(by=['error'], ascending=False).head(25)
top_smallest_error = test_data.sort_values(by=['error'], ascending=True).head(25)

In [ ]:
top_largest_error

In [ ]:
top_smallest_error

In [ ]:
model.save_model("models/b1")